In [45]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io as sio
import glob
from scipy.interpolate import interp1d

In [46]:
data_path = "/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/"
data_path_list = glob.glob(data_path+"*.mat")

mice_ids = np.unique([dp.split("/")[-1][:5] for dp in data_path_list])
session_per_animal = {}
cell_id_per_session = {}
for m_id in mice_ids:
    m_paths_list = glob.glob(data_path + m_id + "*.mat")
    sessions = np.unique([dp.split("/")[-1].split("-")[1][:8] for dp in m_paths_list]).astype(str)
    session_per_animal[m_id] = {}
    for sess in sessions:
        s_paths_list = glob.glob(data_path + m_id + "-" + sess + "*.mat")
        cell_ids = np.unique([dp.split("/")[-1].split(".")[-2][-4:] for dp in s_paths_list]).astype(str)
        session_per_animal[m_id][sess] = {}
        for cell_id in cell_ids:
            r_path = s_paths_list = glob.glob(data_path + m_id + "-" + sess + "*" + cell_id + "*.mat")
            print(r_path[0])
            session_per_animal[m_id][sess][cell_id] = sio.loadmat(r_path[0])
        
recording_paths = sorted(glob.glob(data_path+mice_ids[0]+"-*.mat"))

/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11084-03020501_t2c1.mat
/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11084-03020501_t2c2.mat
/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11084-10030502_t1c1.mat
/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11084-10030502_t1c2.mat
/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11084-10030502_t1c6.mat
/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11084-10030502_t3c7.mat
/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11084-10030502_t4c1.mat
/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11138-11040509_t5c1.mat
/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11207-11060502_t6c2.mat
/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11207-11060502_t6c3.mat
/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11207-11060502_t6c4.mat
/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11207-11060502_t6c5.mat
/hom

In [47]:
data_path_list

['/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11207-21060503_t8c1.mat',
 '/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11343-08120502_t8c2.mat',
 '/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11207-11060502_t6c2.mat',
 '/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11207-16060501_t7c1.mat',
 '/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11084-10030502_t1c2.mat',
 '/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11138-11040509_t5c1.mat',
 '/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11207-11060502_t6c5.mat',
 '/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11084-10030502_t1c6.mat',
 '/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11084-03020501_t2c2.mat',
 '/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11084-03020501_t2c1.mat',
 '/home/rodrigo/HDisk/8F6BE356-3277-475C-87B1-C7A977632DA7_1/11207-11060502_t6c3.mat',
 '/home/rodrigo/HDisk/8F6BE356-3277-475C-87

In [48]:
mouse_data = {}
for r_path in recording_paths:
    trial_id = r_path.split(".")[0][-4:]
    print(trial_id)
    mouse_data[trial_id] = sio.loadmat(r_path)

t2c1
t2c2
t1c1
t1c2
t1c6
t3c7
t4c1


In [87]:
def clean_data(data, keep_headers=False):
    # print(list(data.keys()))
    aux_dict = {}
    for key, val in data.items():
        if isinstance(val, bytes) or isinstance(val, str) or key=="__globals__":
            if keep_headers:
                aux_dict[key] = val
            continue
        else:
            print(len(val))
            if np.isnan(val).any():
                aux_dict[key] = val
            else:
                # Interpolate nans
                x_range = np.linspace(0, 1, num=len(val))
                nan_indexes = np.logical_not(np.isnan(val))[:, 0]
                clean_x = x_range[nan_indexes]
                clean_val = np.array(val)[nan_indexes, 0]
                print(clean_x.shape, clean_val.shape)
                f = interp1d(clean_x, clean_val, kind='cubic')
                aux_dict[key] = f(x_range)
    return aux_dict

In [88]:
clean_data(mouse_data["t1c2"])

685
(685,) (685,)
29949
(29949,) (29949,)
29949
29949
(29949,) (29949,)
29949
29949
(29949,) (29949,)


{'ts': array([  2.10915625,   2.76973958,   5.83001042,   9.03802083,
          9.286125  ,   9.35247917,   9.41435417,   9.49951042,
          9.64435417,   9.78535417,   9.78978125,  10.01869792,
         11.30578125,  11.36832292,  11.41644792,  11.44139583,
         11.46165625,  11.54284375,  12.59764583,  12.81489583,
         16.65363542,  16.7696875 ,  16.85871875,  17.10317708,
         17.12286458,  17.16494792,  17.527875  ,  17.59041667,
         17.609125  ,  17.72878125,  17.82371875,  17.83198958,
         17.88164583,  17.96396875,  17.97798958,  18.01192708,
         18.0759375 ,  18.10465625,  18.18141667,  18.18916667,
         19.41011458,  20.25289583,  20.28913542,  20.30802083,
         20.33691667,  20.35163542,  20.36646875,  20.38335417,
         20.40916667,  20.46414583,  20.47167708,  20.50157292,
         20.52536458,  20.53189583,  20.55232292,  20.57772917,
         20.60186458,  20.62617708,  20.64964583,  20.69385417,
         20.70729167,  20.7220625 